In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.utils import image_dataset_from_directory

In [55]:
train_ds = image_dataset_from_directory(
    "../data/human-and-non-human/training_set/training_set",
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(250,250),
    batch_size=32,
    labels="inferred",
)
train_ds

Found 8017 files belonging to 2 classes.
Using 5612 files for training.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [54]:
val_ds = image_dataset_from_directory(
 "../data/human-and-non-human/training_set/training_set",
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(250,250),
  batch_size=32,
  labels="inferred",
)
val_ds

Found 8017 files belonging to 2 classes.
Using 2405 files for validation.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
from keras import Sequential
from keras import layers
from keras.layers import Rescaling

In [56]:

from keras import layers, models

# Load pretrained MobileNetV2
base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))

# Freeze the base model (not trainable)
base_model.trainable = False

# Add custom layers on top for your specific task
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification: human (1) vs not human (0)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Now, you can fine-tune it or train on your human vs not-human dataset
model.fit(train_ds, epochs=5, validation_data=val_ds)


C:\Users\HP\AppData\Local\Temp\ipykernel_13808\540324685.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))


Epoch 1/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 134s 741ms/step - accuracy: 0.8858 - loss: 0.2568 - val_accuracy: 0.9609 - val_loss: 0.1045
Epoch 2/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 137s 782ms/step - accuracy: 0.9701 - loss: 0.0805 - val_accuracy: 0.9696 - val_loss: 0.0745
Epoch 3/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 146s 830ms/step - accuracy: 0.9844 - loss: 0.0482 - val_accuracy: 0.9613 - val_loss: 0.0927
Epoch 4/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 119s 676ms/step - accuracy: 0.9880 - loss: 0.0369 - val_accuracy: 0.9742 - val_loss: 0.0671
Epoch 5/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 78s 441ms/step - accuracy: 0.9916 - loss: 0.0291 - val_accuracy: 0.9825 - val_loss: 0.0495


In [59]:

test_training_ds = image_dataset_from_directory(
  "../data/human-and-non-human/test_set/test_set",
  seed=123,
  image_size=(250,250),
  batch_size=32,
  labels="inferred",

)
test_training_ds

Found 2723 files belonging to 2 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [60]:
model.evaluate(test_training_ds)

86/86 ━━━━━━━━━━━━━━━━━━━━ 27s 308ms/step - accuracy: 0.9737 - loss: 0.0705


[0.06365882605314255, 0.977230966091156]

In [61]:
train_ds.class_names

['humans', 'non-humans']

In [71]:
import cv2 as cv
import numpy as np

# Load grayscale image
image = cv.imread("../data/human-and-non-human/sarbesh.jpg")

# Resize and normalize
resized_image = cv.resize(image, (250, 250))
resized_image = np.array(resized_image)

resized_image = np.expand_dims(resized_image , axis=0)

# Now this will work
prediction = model.predict(resized_image)
if prediction>0.5:
    print("human")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
human


In [68]:
model.predict(resized_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


array([[0.97927094]], dtype=float32)

In [73]:
import joblib
joblib.dump(model,"human_recog_model.joblib")

['human_recog_model.joblib']

In [11]:
import pandas as pd
import random

# Sample hospital names
hospital_names = [
    "Amri Hospital", "City Hospital", "Medilane Hospital",
    "Care Hospital", "Sunrise Hospital", "LifeCare Hospital",
    "Apollo Hospital", "Healing Touch Clinic", "Global Health Centre",
    "Green Cross Hospital"
]

# Blood groups
blood_groups = ["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"]

# Urgency levels
urgency_levels = ["Regular Need", "URGENT"]

# Generate synthetic data
data = []
for _ in range(200):
    hospital = random.choice(hospital_names)
    blood_group = random.choice(blood_groups)
    urgency = random.choice(urgency_levels)
    units_needed = random.randint(1, 10)
    
    data.append([hospital, blood_group, urgency, units_needed])

# Create DataFrame
df = pd.DataFrame(data, columns=["Hospital Name", "Blood Group", "Urgency", "Units Needed"])

print("Original Data:")
print(df.head())

# Encoding
# 1. Label Encoding the Hospital Name
df["Hospital Name Encoded"] = df["Hospital Name"].astype('category').cat.codes

# 2. Label Encoding the Blood Group
df["Blood Group Encoded"] = df["Blood Group"].astype('category').cat.codes

# 3. Encoding Urgency (0 for Regular, 1 for URGENT)
df["Urgency Encoded"] = df["Urgency"].map({"Regular Need": 0, "URGENT": 1})

# Now the preprocessed dataframe
preprocessed_df = df[[
    "Hospital Name Encoded", "Blood Group Encoded", "Urgency Encoded", "Units Needed"
]]

print("\nPreprocessed Data (Encoded):")
print(preprocessed_df.head())

# Create a 'Shortage' label based on urgency and units needed
df['Shortage'] = df.apply(
    lambda row: 1 if (row['Urgency'] == "URGENT" and row['Units Needed'] > 5) else 0, 
    axis=1
)

print(df[['Blood Group', 'Urgency', 'Units Needed', 'Shortage']].head())
df


Original Data:
          Hospital Name Blood Group       Urgency  Units Needed
0         Care Hospital          B+        URGENT             3
1  Global Health Centre          B+  Regular Need             7
2  Healing Touch Clinic          A-        URGENT            10
3         Care Hospital          B-        URGENT            10
4     Medilane Hospital         AB-        URGENT             5

Preprocessed Data (Encoded):
   Hospital Name Encoded  Blood Group Encoded  Urgency Encoded  Units Needed
0                      2                    4                1             3
1                      4                    4                0             7
2                      6                    1                1            10
3                      2                    5                1            10
4                      8                    3                1             5
  Blood Group       Urgency  Units Needed  Shortage
0          B+        URGENT             3         0
1    

,Hospital Name,Blood Group,Urgency,Units Needed,Hospital Name Encoded,Blood Group Encoded,Urgency Encoded,Shortage
0,Care Hospital,B+,URGENT,3,2,4,1,0
1,Global Health Centre,B+,Regular Need,7,4,4,0,0
2,Healing Touch Clinic,A-,URGENT,10,6,1,1,1
3,Care Hospital,B-,URGENT,10,2,5,1,1
4,Medilane Hospital,AB-,URGENT,5,8,3,1,0
...,...,...,...,...,...,...,...,...
195,LifeCare Hospital,A+,Regular Need,1,7,0,0,0
196,LifeCare Hospital,A+,URGENT,4,7,0,1,0
197,Green Cross Hospital,AB+,Regular Need,4,5,2,0,0
198,Care Hospital,A-,URGENT,3,2,1,1,0


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
X = np.array(df[['Urgency Encoded' , 'Units Needed']])
y = np.array(df['Shortage'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


shortage_predict_model = RandomForestClassifier()
shortage_predict_model.fit(X_train, y_train)


print("Training accuracy:", shortage_predict_model.score(X_train, y_train))
print("Testing accuracy:", shortage_predict_model.score(X_test, y_test))


Training accuracy: 1.0
Testing accuracy: 1.0


In [31]:
import joblib
joblib.dump(shortage_predict_model , "shortage_predict_model.joblib")

['shortage_predict_model.joblib']